In [1]:
pip install accelerate -U

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/53/fe/0251ccd9e0015c705e772da0fb2c96cdafd87b1d7dd45dc13dca7ced0eb7/accelerate-0.29.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 307.3 kB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install transformers -U


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/09/c8/844d5518a6aeb4ffdc0cf0cae65ae13dbe5838306728c5c640b5a6e2a0c9/transformers-4.40.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 327.2 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Obtaining dependency information for huggingface-hub<1.0,>=0.19.3 from https://files.pythonhosted.org/packages/05/c0/779afbad8e75565c09ffa24a88b5dd7e293c92b74eb09df6435fc58ac986/huggingface_hub-0.22.2-py3-none-any.whl.metadata
  Obtaining dependency information for tokenizers<0.20,>=0.19 from https://files.pythonhosted.org/packages/a7/03/fb50fc03f86016b227a967c8d474f90230c885c0d18f78acdfda7a96ce56/tokenizers-0.19.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for safetensors>=0.4.1 from https://files.pythonhosted.org/packages/d5/85/1e7d2804cbf82204cde462d16f1cb0ff5814b03f559fb46ceaa6b7020db4/safetensors-0.4.3-cp311-cp311-

In [1]:
import pandas as pd

In [2]:
# https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/data
data = pd.read_csv("english_preprocessed_exploded.csv")
data.head()

,English_processed,label_task1,label_task2,label_task3
0,Ignore the other one hes an asshole The probl...,YES,REPORTED,OBJECTIFICATION
1,Ignore the other one hes an asshole The probl...,YES,JUDGEMENTAL,OBJECTIFICATION
2,Ignore the other one hes an asshole The probl...,YES,JUDGEMENTAL,SEXUAL-VIOLENCE
3,Ignore the other one hes an asshole The probl...,NO,-,-
4,Ignore the other one hes an asshole The probl...,YES,REPORTED,STEREOTYPING-DOMINANCE


In [3]:
# data.to_csv("/content/drive/MyDrive/Youtube Tutorials/datasets/toxic_commnets.csv",index=False)

In [4]:
data = data.drop(data[data['label_task2'] == "-"].index)


In [ ]:
data['label_task2'].value_counts()

label_task2
DIRECT         12556
JUDGEMENTAL     6587
REPORTED        6225
UNKNOWN           99
Name: count, dtype: int64

In [ ]:
data = data[['English_processed','label_task2']]
data = data[0:1000]
data.head(10)

,English_processed,label_task2
0,Ignore the other one hes an asshole The probl...,REPORTED
1,Ignore the other one hes an asshole The probl...,JUDGEMENTAL
2,Ignore the other one hes an asshole The probl...,JUDGEMENTAL
4,Ignore the other one hes an asshole The probl...,REPORTED
5,Ignore the other one hes an asshole The probl...,JUDGEMENTAL
6,Ignore the other one hes an asshole The probl...,REPORTED
7,Ignore the other one hes an asshole The probl...,REPORTED
12,If comicsgate is anything like gamergate then...,DIRECT
22,A quite unfortunate social delay gamergate tr...,DIRECT
24,A quite unfortunate social delay gamergate tr...,REPORTED


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data["label_task2"]=le.fit_transform(data["label_task2"])

In [ ]:
data["label_task2"]

0       2
1       1
2       1
4       2
5       1
       ..
1956    2
1957    2
1958    2
1959    1
1961    0
Name: label_task2, Length: 1000, dtype: int64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available, using GPU for computation.')
else:
    device = torch.device('cpu')
    print('CUDA is not available, using CPU for computation.')

CUDA is available, using GPU for computation.


In [ ]:
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [ ]:
X = list(data["English_processed"])
y = list(data["label_task2"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [ ]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [ ]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
len(X_train),len(X_val)

(800, 200)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
train_dataset[5]

{'input_ids': tensor([ 101, 2065, 1045, 2020, 2017, 1045, 2052, 2079, 2242, 2085, 2000, 2644,
         2045, 2108, 2061, 2116, 3572, 1997, 5907, 4808, 2114, 2308, 2057, 2031,
         2000, 2681, 2068, 4142, 1998, 2489, 2027, 2024, 2111, 2066, 2149, 2053,
         2062, 5907, 4808,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average=None)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=1,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=100, training_loss=0.5350914382934571, metrics={'train_runtime': 15.2925, 'train_samples_per_second': 52.313, 'train_steps_per_second': 6.539, 'total_flos': 28778288448000.0, 'train_loss': 0.5350914382934571, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [ ]:
np.set_printoptions(suppress=True)

In [ ]:
text = "That was good point"
# text = "go to hell"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.5984,  0.6522, -1.7499, -2.6356]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.8613, 0.1230, 0.0111, 0.0046]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)


array([[0.8612643 , 0.12300754, 0.01113555, 0.00459264]], dtype=float32)

In [ ]:
trainer.save_model('CustomModel')

In [ ]:
# trainer.save_model('/content/drive/MyDrive/Youtube Tutorials/toxic')
# model_2 = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Youtube Tutorials/toxic")
# model_2.to('cuda')

In [ ]:
model_2 = BertForSequenceClassification.from_pretrained("CustomModel")
model_2.to('cuda')

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
# text = "That was good point"
text = "     My lions putting balls in pussy       "
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model_2(**inputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
predictions

array([[0.99667203, 0.00111366, 0.00107008, 0.00114426]], dtype=float32)

In [ ]:

index = np.argmax(predictions, axis=1)
print(index.dtype)

int64


In [ ]:
reverse_mapping = dict(zip(le.transform(le.classes_), le.classes_))


In [ ]:
predicted_class_index = ...  # Replace this with the predicted class index from your model
original_label = reverse_mapping[0]
original_label

'DIRECT'

# Pipeline

In [ ]:
myModel = BertForSequenceClassification.from_pretrained("CustomModel")
myModel.to('cuda')
def getType(input_text):
  inputs = tokenizer(input_text,padding = True, truncation = True, return_tensors='pt').to('cuda')
  outputs = myModel(**inputs)
  predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
  predictions = predictions.cpu().detach().numpy()
  index = np.argmax(predictions, axis=1)
  return reverse_mapping[index[0]]


In [ ]:
getType(" You got it Lalu  The cover went from liking it a lot to not liking it at all  Seen next to Messiswell yes  I tell you that I identify with her a thousand times more in these photos  But that is not the point  Yes it is microaggression")

'REPORTED'

In [ ]:
sample_data=data.head(10)
sample_data["pred"]=sample_data.apply(lambda x: getType(x["English_processed"]))

KeyError: 'English_processed'

In [ ]:
data["pred"]=data.apply(lambda x: getType(x["English_processed"]))

,English_processed,label_task2
0,Ignore the other one hes an asshole The probl...,2
1,Ignore the other one hes an asshole The probl...,1
2,Ignore the other one hes an asshole The probl...,1
4,Ignore the other one hes an asshole The probl...,2
5,Ignore the other one hes an asshole The probl...,1
